In [1]:
import pandas as pd
import datetime as dt

In [2]:
tablero_vacantes= pd.read_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\Actualizacion de informacion\Vacantes 2025.xlsx")
tablero_aps = pd.read_excel(r"C:\Users\centro.control\Downloads\TableroAps-2025-10-27.xlsx ", skiprows=[1])
nuevos_nombres = pd.read_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\Sitios SAP.xlsx")

In [3]:
criterio_aps =(pd.to_datetime(tablero_aps['FECHA'].max())).date()
criterio_tablero =  (tablero_vacantes['Fecha del reporte'].max()).date()


In [4]:
nombre_sap = nuevos_nombres[['SITIO','NOMBRE COMERCIAL','id','tipo','EJECUTIVO OPS 2025']].rename(columns={'SITIO': 'Nombre Comercial', 'NOMBRE COMERCIAL': 'SITIO'})

In [5]:
tablero_aps = tablero_aps.merge(nombre_sap, on='SITIO', how='left')
tablero_aps['SITIO'] = tablero_aps['Nombre Comercial']
tablero_aps= tablero_aps.drop(columns=['Nombre Comercial'])

In [6]:
def tabla_turno(turno_str):
    # Determinar índice y nombre de columna
    if turno_str == 'Matutino':
        i = 6
        aux = 'MAT'
        turno= 'Matutino'

    elif turno_str == 'Vespertino':
        i = 9
        aux = 'Vesp'
        turno= 'Vespertino'

    elif turno_str == 'Nocturno':
        i = 12
        aux = 'Noc'
        turno= 'Nocturno'

    elif turno_str == 'Mixto':
        i = 15
        aux = 'Mix'
        turno= 'Mixto'

    else:
        raise ValueError("Turno no válido. Usa 'Matutino', 'Vespertino', 'Nocturno' o 'Mixto'.")

    # Selección y renombramiento de columnas
    columnas = ['id','FECHA', 'SITIO',  aux, f'Unnamed: {i}', f'Unnamed: {i+1}','tipo', 'EJECUTIVO OPS 2025']
    turno_df = tablero_aps[columnas].copy()

    turno_df = turno_df.rename(columns={
        'FECHA': 'Fecha del reporte',
        'SITIO': 'Sitio',
        aux: 'Plantilla',
        f'Unnamed: {i}': 'Activos',
        f'Unnamed: {i+1}': 'Vacante',
        'tipo': 'Tipo',
        'EJECUTIVO OPS 2025': 'Ejecutivo'
    })

    turno_df['Turno'] = turno

    return turno_df

In [7]:
Matutino = tabla_turno('Matutino')
Vespertino = tabla_turno('Vespertino')
Nocturno = tabla_turno('Nocturno')
Mixto = tabla_turno('Mixto')

In [8]:
tablero_actualizado = pd.concat([tablero_vacantes, Matutino, Vespertino, Nocturno, Mixto], ignore_index=True)

In [9]:
tablero_actualizado['Fecha del reporte'] = pd.to_datetime(tablero_actualizado['Fecha del reporte'])
tablero_actualizado['Fecha del reporte'] = tablero_actualizado['Fecha del reporte'].dt.date
tablero_actualizado = tablero_actualizado.drop_duplicates()

In [10]:
tablero_actualizado

,id,Fecha del reporte,Sitio,Plantilla,Activos,Vacante,Turno,Tipo,Ejecutivo
0,241,2025-03-01,LIVERPOOL PERINORTE,14,11,3,Matutino,Natural,Karina Sánchez
1,358,2025-03-01,AMERICAN EXPRESS POLANCO,14,15,0,Matutino,Natural,Ángel Cervantes
2,357,2025-03-01,LIVERPOOL JILOTEPEC,13,13,0,Matutino,Natural,Oscar Castillo
3,183,2025-03-01,LIVERPOOL INTERLOMAS,27,27,0,Matutino,Natural,Abel Vázquez
4,80,2025-03-01,CUMBRES PRIMARIA Y KINDER,7,8,0,Matutino,Natural,Ángel Cervantes
...,...,...,...,...,...,...,...,...,...
97503,254,2025-10-27,LIVERPOOL CIUDAD JARDIN,0,0,0,Mixto,NATURAL,Karina Sánchez
97504,255,2025-10-27,SUBURBIA SATELITE,0,0,0,Mixto,NATURAL,Karina Sánchez
97505,256,2025-10-27,ENCUENTRO OCEANIA,1,1,0,Mixto,NATURAL,Guillermina Vázquez
97506,257,2025-10-27,ASSA ABLOY PELICANO,0,0,0,Mixto,NATURAL,Karina Sánchez


In [11]:
if criterio_aps == criterio_tablero: 
        print('No se pudo actualizar la información')
else:
        tablero_actualizado.to_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\Actualizacion de informacion\Vacantes 2025.xlsx", index=False)

In [12]:
import psutil
import os
process = psutil.Process(os.getpid())
memory_info = process.memory_info()
print(f"Uso de memoria: {memory_info.rss / 1024 / 1024:.2f} MB")

Uso de memoria: 386.24 MB
